# Submit study data to GEO
This Python Jupyter notebook submits the processed and raw study data to the [GEO database](https://www.ncbi.nlm.nih.gov/geo/).

## Import Python modules

In [1]:
import os
import shutil
import glob
import hashlib
import ftplib

import openpyxl

## Read the Excel metadata template
We have already created an Excel metadata workbook by manually filling the [GEO template](https://www.ncbi.nlm.nih.gov/geo/info/examples/seq_template_v2.1.xls) with information appropriate for our experiment.

We read this notebook and get the single active worksheet.
This sheet has all the relevant information **except** the MD5 checksums.

In [2]:
# read metadata template Excel workbook
wb = openpyxl.load_workbook('metadata_template.xlsx')

# get active worksheet, which should be first and only one
assert len(wb.sheetnames) == 1, "multiple notebook sheets"
ws = wb.active

## Convert CCS BAM files to FASTQ
The PacBio CCS files are currently stored as `*ccs.bam` files as generated by the `ccs` program.
We want to convert them to gzipped FASTQ to upload, and those are what are specified in our mdetadata template.

So here we use `samtools` to do this, keeping the tags that give the accuracy and number of passes:

In [3]:
ccsdir = '../results/pacbio/ccs/'

for bam_ccs in glob.glob(f'{ccsdir}/*ccs.bam'):
    fastq_ccs = os.path.splitext(bam_ccs)[0] + '.fastq.gz'
    if not os.path.isfile(fastq_ccs):
        print(f"Creating {fastq_ccs} from {bam_ccs}")
        !samtools bam2fq -T np,rq {bam_ccs} | gzip > {fastq_ccs} 
        assert os.path.isfile(fastq_ccs)
    else:
        print(f"{fastq_ccs} already exists")

../results/pacbio/ccs/2017-06-08_ccs.fastq.gz already exists
../results/pacbio/ccs/2018-08-08_Pol_circ_ccs.fastq.gz already exists
../results/pacbio/ccs/2018-06-22_Pol-1_ccs.fastq.gz already exists
../results/pacbio/ccs/2018-06-22_nonPol_ccs.fastq.gz already exists
../results/pacbio/ccs/2018-06-22_Pol_open_ccs.fastq.gz already exists
../results/pacbio/ccs/2018-06-22_Pol-2_ccs.fastq.gz already exists
../results/pacbio/ccs/2017-12-07_ccs.fastq.gz already exists


## Get MD5 checksums and files to upload
We want to parse the worksheet to identify all files that need to be submitted to GEO and then do the following:
 1. Add the MD5 checksum to the worksheet
 2. Add the file to the list to upload to GEO
 
These needs to be done for two titled sections of the notebook:
 - a section titled *PROCESSED DATA FILES*
 - a section titled *RAW FILES*
 
Each of these sections should first have a heading line with the first three columns being *file name*, *file type*, and *file checksum*--with the last of these columns where we add the checksum.
Each section ends with the first line that is a comment (begins with `#`).

So we go through the notebook and create a dict that is keyed by the names of all processed and raw data files that we need to upload, and has as its values the cell in the worksheet where the MD5 checksum for that file needs to be placed:

In [4]:
rows = list(ws.rows)

files_to_upload = {}

for section in ['PROCESSED DATA FILES', 'RAW FILES']:
    
    print(f"\nIdentifying {section} files:")
    
    # identify row with section title
    titlecell = [row[0] for row in rows if row[0].value == section]
    if len(titlecell) != 1:
        raise ValueError(f"not exactly one title row for {section}")
    titlerow = titlecell[0].row
    
    # check correctness of header following section title
    header = [cell.value for cell in rows[titlerow][ : 3]]
    if header != ['file name', 'file type', 'file checksum']:
        raise ValueError(f"header incorrect for {section}")
        
    # get all rows with files to upload
    i = 1 + titlerow
    filename = rows[i][0].value
    while ((filename is not None) and (filename[0] != '#') and not filename.isspace()):
        print(filename)
        checksumcell = rows[i][2]
        if checksumcell.value is not None:
            raise ValueError("checksum cell already contains a value")
        files_to_upload[filename] = checksumcell
        i += 1
        filename = rows[i][0].value


Identifying PROCESSED DATA FILES files:
merged_canine_cells.tsv
merged_canine_genes.tsv
merged_canine_matrix.mtx
merged_humanplusflu_cells.tsv
merged_humanplusflu_genes.tsv
merged_humanplusflu_matrix.mtx
PacBio_annotated_merged_humanplusflu_cells.tsv

Identifying RAW FILES files:
2017-06-08_ccs.fastq.gz
2017-06-08_report.csv
2017-12-07_ccs.fastq.gz
2017-12-07_report.csv
2018-06-22_nonPol_ccs.fastq.gz
2018-06-22_nonPol_report.csv
2018-06-22_Pol-1_ccs.fastq.gz
2018-06-22_Pol-1_report.csv
2018-06-22_Pol-2_ccs.fastq.gz
2018-06-22_Pol-2_report.csv
2018-06-22_Pol_open_ccs.fastq.gz
2018-06-22_Pol_open_report.csv
2018-08-08_Pol_circ_ccs.fastq.gz
2018-08-08_Pol_circ_report.csv
IFN_enriched_S1_L002_R1_001.fastq.gz
IFN_enriched_S1_L002_R2_001.fastq.gz
IFN_enriched_S1_L002_I1_001.fastq.gz


Now we look for each of these files.
We specify directories where they are stored, and make sure each file is uniquely located in one of these directories:

In [5]:
# directories where files may be found
search_dirs = [
        # location of Illumina FASTQ files for transcriptomics
        '../results/demultiplexed_reads/2017-07-21/fastq/IFN_enriched/',
        # location of PacBio CCS files
        ccsdir,
        # location of cell-gene matrix files
        '../results/cellgenecounts/'
        ]

search_files = []
for search_dir in search_dirs:
    search_files += glob.glob(f'{search_dir}/*')
    
# get full path to each file
fullpaths = {}
for fbase, cell in files_to_upload.items():
    for fullf in search_files:
        if os.path.basename(fullf) == os.path.basename(fbase):
            if fbase not in fullpaths:
                fullpaths[fbase] = fullf
            else:
                raise ValueError(f"Found multiple occurrences of {fbase}")
    if fbase not in fullpaths:
        raise ValueError(f"Failed to find full path for {fbase}")

Now we compute the MD5 checksum for each file and fill back into the Excel worksheet:

In [6]:
# copied from here: https://stackoverflow.com/a/11143944
def md5sum(filename):
    md5 = hashlib.md5()
    with open(filename, 'rb') as f:
        for chunk in iter(lambda: f.read(128 * md5.block_size), b''):
            md5.update(chunk)
    return md5.hexdigest()

for fbase, fullf in fullpaths.items():
    md5checksum = md5sum(fullf)
    print(f"MD5 checksum for {fbase} is {md5checksum}")
    files_to_upload[fbase].value = md5checksum

MD5 checksum for merged_canine_cells.tsv is 44f0cc89e99d228ad8779caee0c7fd20
MD5 checksum for merged_canine_genes.tsv is 4aebdd28ed1a69be3153e8b683cedaf6
MD5 checksum for merged_canine_matrix.mtx is a77c1dad0ab19d0ab1fba006a3bc8b2d
MD5 checksum for merged_humanplusflu_cells.tsv is 726c40311766d48f0bf15deea86c4d30
MD5 checksum for merged_humanplusflu_genes.tsv is 3676d0cbd61565f4712b442d692a4ca7
MD5 checksum for merged_humanplusflu_matrix.mtx is c7abf72c769f7e8c26f104683a162de3
MD5 checksum for PacBio_annotated_merged_humanplusflu_cells.tsv is ed70abd15bca32bc373d1f656704b5cb
MD5 checksum for 2017-06-08_ccs.fastq.gz is e19c98ff793be4685595b584531ed36b
MD5 checksum for 2017-06-08_report.csv is 26d1727811f15b380fc0bd26630e8898
MD5 checksum for 2017-12-07_ccs.fastq.gz is 8fde9acce7dc5f4db84b6b8a9fbb36c3
MD5 checksum for 2017-12-07_report.csv is f3ef600db9558a3325ac7e1c02514778
MD5 checksum for 2018-06-22_nonPol_ccs.fastq.gz is 5674846b3243c125af2245b2a84bd7d0
MD5 checksum for 2018-06-22_no

Now save the Excel workbook with the filled values to a new name:

In [7]:
completed_metadata = 'metadata.xlsx'
wb.save(filename=completed_metadata)

print(f"{completed_metadata} is now an Excel workbook with the completed metadata.")

metadata.xlsx is now an Excel workbook with the completed metadata.


## Now upload files to GEO
We now upload the files to GEO via FTP following the [instructions here](https://www.ncbi.nlm.nih.gov/geo/info/submissionftp.html).

Note that this requires a username and password.
For security reasons, those are not in this notebook but are in a file called `GEO_password.txt` in this directory (but **not** tracked on GitHub) with the first line being the username and the second being the password.
In order to run this notebook, you need to create this file yourself with a valid username and password:

In [8]:
with open('GEO_password.txt') as f:
    lines = f.readlines()
    if len(lines) != 2:
        raise ValueError("Did not find exactly two lines")
    username = lines[0].strip()
    password = lines[1].strip()

Now we connect to the FTP server:

In [9]:
ftp = ftplib.FTP(host='ftp-private.ncbi.nlm.nih.gov',
                 user=username,
                 passwd=password)

Next, we create a directory with our GEO username as instructed on the website.
For me, this is jbloom@fhcrc.org.
We then transfer all the files to this directory, and print the contents of the directory after transfer.
Finally, we close the connection.

In [10]:
dirname = 'jbloom@fhcrc.org'
if dirname not in ftp.nlst():
    ftp.mkd(dirname)
ftp.cwd(dirname)

with open(completed_metadata, 'rb') as f:
    ftp.storbinary('STOR metadata.xlsx', f)
for fbase, fullf in fullpaths.items():
    with open(fullf,  'rb') as f:
        ftp.storbinary(f'STOR {fbase}', f)

print(f"Here are files transferred to directory {ftp.pwd()}:")
print(ftp.retrlines('LIST'))

ftp.close()

Here are files transferred to directory /jbloom@fhcrc.org:
-rw-rw-r--   1 geo      geo       8876563 Oct 26 18:47 2017-06-08_ccs.fastq.gz
-rw-rw-r--   1 geo      geo           832 Oct 26 18:47 2017-06-08_report.csv
-rw-rw-r--   1 geo      geo      64380085 Oct 26 18:47 2017-12-07_ccs.fastq.gz
-rw-rw-r--   1 geo      geo           822 Oct 26 18:47 2017-12-07_report.csv
-rw-rw-r--   1 geo      geo      51120866 Oct 26 18:47 2018-06-22_Pol-1_ccs.fastq.gz
-rw-rw-r--   1 geo      geo           818 Oct 26 18:47 2018-06-22_Pol-1_report.csv
-rw-rw-r--   1 geo      geo      60266203 Oct 26 18:48 2018-06-22_Pol-2_ccs.fastq.gz
-rw-rw-r--   1 geo      geo           819 Oct 26 18:48 2018-06-22_Pol-2_report.csv
-rw-rw-r--   1 geo      geo      69642418 Oct 26 18:48 2018-06-22_Pol_open_ccs.fastq.gz
-rw-rw-r--   1 geo      geo           820 Oct 26 18:48 2018-06-22_Pol_open_report.csv
-rw-rw-r--   1 geo      geo      66592996 Oct 26 18:47 2018-06-22_nonPol_ccs.fastq.gz
-rw-rw-r--   1 geo      geo      

## Notify GEO that data have been transferred!
Finally, remember to notify GEO that the data have been transferred [as described here](https://submit.ncbi.nlm.nih.gov/geo/submission/).